In [1]:
import os
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from pyimzml.ImzMLParser import ImzMLParser
from tqdm import tqdm
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from datetime import datetime
import gc

In [2]:
# Define folder that contains the preprocessed dhg dataset
DHG_IN_PATH = "/sise/assafzar-group/assafzar/Leor/DHG/Preprocessed"
# Define file to export
META_DATA_PATH = "/sise/assafzar-group/assafzar/Leor/DHG/Preprocessed/Metadata.csv"

In [3]:
# 
meta_data = pd.read_csv(META_DATA_PATH)
# 
meta_data = meta_data[meta_data.file_name.str.contains('s')]
# 
meta_data["label"] = (meta_data.who_grade > 2).astype(int)

In [4]:
#
parsers = { file_name: ImzMLParser(os.path.join(DHG_IN_PATH, f"{file_name}.imzML")) for file_name in meta_data.file_name.unique()}

In [5]:
class DataGenerator(keras.utils.Sequence):
  'Generates data for Keras'
  def __init__(self, df, parsers, batch_size=32,
               dim=(92000,), shuffle=True):
    'Initialization'
    self.dim = dim
    self.batch_size = batch_size
    self.df = df
    self.parsers = parsers
    self.shuffle = shuffle
    self.on_epoch_end()

  def __len__(self):
    'Denotes the number of batches per epoch'
    return int(np.floor(len(self.df) / self.batch_size))

  def __getitem__(self, index):
    'Generate one batch of data'
    # Generate indexes of the batch
    batch_df = self.df[index*self.batch_size:(index+1)*self.batch_size]
    # Generate data
    X, y = self.__data_generation(batch_df)
    return X, y

  def on_epoch_end(self):
    'Updates indexes after each epoch'
    if self.shuffle == True:
        self.df = self.df.sample(frac=1).reset_index(drop=True)

  def __data_generation(self, batch_df):
    'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
    # Initialization
    X = np.empty((self.batch_size, *self.dim))
    y = np.empty((self.batch_size), dtype=int)
    # Generate data
    for i, (_, row) in enumerate(batch_df.iterrows()):
      # get spectra
      _, spectra = self.parsers[row.file_name].getspectrum(row.idx)
      # Store sample
      X[i,] = spectra
      # Store class
      y[i] = row.label
    return X, y

In [ ]:
#
now = datetime.now()
#
with open(f"/sise/assafzar-group/assafzar/Leor/log {now.strftime('%d-%m-%Y-%H-%M-%S')}.txt", "w") as file:
  #
  for exclude_sample in tqdm(meta_data.file_name.unique()):
    #
    gc.collect()
    # 
    meta_data_train, meta_data_val = train_test_split(
      meta_data[meta_data.file_name != exclude_sample], test_size=0.2)
    # 
    meta_data_test = meta_data[meta_data.file_name == exclude_sample]

    # Parameters
    params = {"dim": (92000,), "batch_size": 512,
              "shuffle": True, "parsers": parsers}

    # Generators
    training_generator = DataGenerator(meta_data_train, **params)
    validation_generator = DataGenerator(meta_data_val, **params)
    testing_generator = DataGenerator(meta_data_test, **params)

    #
    model = keras.Sequential(
        [
          keras.Input(shape=(92000,)),
          layers.Dense(512, activation='relu'),
          layers.Dense(512, activation='relu'),
          layers.Dense(256, activation='relu'),
          layers.Dense(256, activation='relu'),
          layers.Dense(128, activation='relu'),
          layers.Dense(1, activation='sigmoid')
        ]
    )
    # 
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    # 
    history = model.fit(x=training_generator, validation_data=validation_generator, epochs=10)
    # 
    test_eval = model.evaluate(testing_generator)
    
    #
    file.write(f"Excluded: {exclude_sample}\n")
    file.write(f"Train loss: {history.history['loss']}\n")
    file.write(f"Train accuracy: {history.history['accuracy']}\n")
    file.write(f"Validation loss: {history.history['val_loss']}\n")
    file.write(f"Validation accuracy: {history.history['val_accuracy']}\n")
    file.write(f"Test loss: {test_eval[0]}\n")
    file.write(f"Test accuracy: {test_eval[1]}\n\n")

  0%|          | 0/24 [00:00<?, ?it/s]2022-04-23 18:26:55.608590: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-04-23 18:26:55.608674: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-04-23 18:26:55.608722: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (cs-cpu-22): /proc/driver/nvidia/version does not exist
2022-04-23 18:26:55.611911: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-04-23 18:27:06.715414: I tensorflow/compiler/mlir/mlir_graph_

Epoch 1/10
41/41 [==============================] - 284s 7s/step - loss: 0.6537 - accuracy: 0.6174 - val_loss: 0.5669 - val_accuracy: 0.6113
Epoch 2/10
41/41 [==============================] - 183s 4s/step - loss: 0.3596 - accuracy: 0.8321 - val_loss: 0.2178 - val_accuracy: 0.9248
Epoch 3/10
41/41 [==============================] - 180s 4s/step - loss: 0.1514 - accuracy: 0.9491 - val_loss: 0.0611 - val_accuracy: 0.9791
Epoch 4/10
41/41 [==============================] - 174s 4s/step - loss: 0.0183 - accuracy: 0.9940 - val_loss: 0.0110 - val_accuracy: 0.9965
Epoch 5/10
41/41 [==============================] - 174s 4s/step - loss: 0.0016 - accuracy: 0.9998 - val_loss: 0.0110 - val_accuracy: 0.9965
Epoch 6/10
41/41 [==============================] - 173s 4s/step - loss: 7.2963e-04 - accuracy: 0.9999 - val_loss: 0.0078 - val_accuracy: 0.9975
Epoch 7/10
41/41 [==============================] - 182s 4s/step - loss: 1.5200e-04 - accuracy: 1.0000 - val_loss: 0.0083 - val_accuracy: 0.9979
Epoch

  4%|▍         | 1/24 [31:37<12:07:19, 1897.35s/it]

Epoch 1/10
41/41 [==============================] - 177s 4s/step - loss: 0.6331 - accuracy: 0.6270 - val_loss: 0.4780 - val_accuracy: 0.6164
Epoch 2/10
41/41 [==============================] - 178s 4s/step - loss: 0.3254 - accuracy: 0.8777 - val_loss: 0.2244 - val_accuracy: 0.9303
Epoch 3/10
41/41 [==============================] - 185s 4s/step - loss: 0.1802 - accuracy: 0.9383 - val_loss: 0.2076 - val_accuracy: 0.9410
Epoch 4/10
41/41 [==============================] - 193s 5s/step - loss: 0.0480 - accuracy: 0.9855 - val_loss: 0.0171 - val_accuracy: 0.9934
Epoch 5/10
41/41 [==============================] - 181s 4s/step - loss: 0.0059 - accuracy: 0.9980 - val_loss: 0.0072 - val_accuracy: 0.9967
Epoch 6/10
41/41 [==============================] - 179s 4s/step - loss: 0.0019 - accuracy: 0.9996 - val_loss: 0.0050 - val_accuracy: 0.9980
Epoch 7/10
41/41 [==============================] - 184s 4s/step - loss: 0.0014 - accuracy: 0.9995 - val_loss: 0.0099 - val_accuracy: 0.9963
Epoch 8/10
41

  8%|▊         | 2/24 [1:01:57<11:27:11, 1874.17s/it]

...

Epoch 1/10
41/41 [==============================] - 182s 4s/step - loss: 0.6372 - accuracy: 0.6214 - val_loss: 0.4796 - val_accuracy: 0.6307
Epoch 2/10
41/41 [==============================] - 180s 4s/step - loss: 0.3484 - accuracy: 0.8692 - val_loss: 0.2257 - val_accuracy: 0.9273
Epoch 3/10
41/41 [==============================] - 181s 4s/step - loss: 0.1356 - accuracy: 0.9561 - val_loss: 0.0500 - val_accuracy: 0.9867
Epoch 4/10
41/41 [==============================] - 181s 4s/step - loss: 0.0186 - accuracy: 0.9941 - val_loss: 0.0161 - val_accuracy: 0.9951
Epoch 5/10
41/41 [==============================] - 180s 4s/step - loss: 0.0021 - accuracy: 0.9994 - val_loss: 0.0113 - val_accuracy: 0.9961
Epoch 6/10
41/41 [==============================] - 182s 4s/step - loss: 4.3536e-04 - accuracy: 1.0000 - val_loss: 0.0099 - val_accuracy: 0.9973
Epoch 7/10
41/41 [==============================] - 182s 4s/step - loss: 1.6906e-04 - accuracy: 1.0000 - val_loss: 0.0090 - val_accuracy: 0.9973
Epoch

 79%|███████▉  | 19/24 [9:24:37<2:29:20, 1792.06s/it]

Epoch 1/10
41/41 [==============================] - 180s 4s/step - loss: 0.6469 - accuracy: 0.6247 - val_loss: 0.5213 - val_accuracy: 0.6264
Epoch 2/10
41/41 [==============================] - 180s 4s/step - loss: 0.3506 - accuracy: 0.8412 - val_loss: 0.2027 - val_accuracy: 0.9322
Epoch 3/10
41/41 [==============================] - 179s 4s/step - loss: 0.1906 - accuracy: 0.9347 - val_loss: 0.1461 - val_accuracy: 0.9520
Epoch 4/10
41/41 [==============================] - 178s 4s/step - loss: 0.0513 - accuracy: 0.9847 - val_loss: 0.0175 - val_accuracy: 0.9941
Epoch 5/10
41/41 [==============================] - 180s 4s/step - loss: 0.0081 - accuracy: 0.9971 - val_loss: 0.0205 - val_accuracy: 0.9937
Epoch 6/10
41/41 [==============================] - 185s 4s/step - loss: 0.0060 - accuracy: 0.9984 - val_loss: 0.0260 - val_accuracy: 0.9924
Epoch 7/10
41/41 [==============================] - 183s 4s/step - loss: 9.7081e-04 - accuracy: 0.9998 - val_loss: 0.0056 - val_accuracy: 0.9982
Epoch 8/1

 83%|████████▎ | 20/24 [9:54:52<1:59:55, 1798.92s/it]

Epoch 1/10
40/41 [============================>.] - ETA: 3s - loss: 0.6445 - accuracy: 0.6280

In [33]:
output = []

with open(f"/sise/assafzar-group/assafzar/Leor/log {now.strftime('%d-%m-%Y-%H-%M-%S')}.txt", "r") as file:
  for _ in tqdm(meta_data.file_name.unique()):
    exclude_sample = file.readline().replace("Excluded: ", "").replace("\n","")
    train_loss = list(file.readline().replace("Train loss: ", "").replace("[", "").replace("]", "").replace("\n","").split(","))[-1]
    train_accuracy = list(file.readline().replace("Train accuracy: ", "").replace("[", "").replace("]", "").replace("\n","").split(","))[-1]
    val_loss = list(file.readline().replace("Validation  loss: ", "").replace("[", "").replace("]", "").replace("\n","").split(","))[-1]
    val_accuracy = list(file.readline().replace("Validation accuracy: ", "").replace("[", "").replace("]", "").replace("\n","").split(","))[-1]
    test_loss = file.readline().replace("Test loss: ", "").replace("\n","")
    test_accuracy = file.readline().replace("Test accuracy: ", "").replace("\n","")
    _ = file.readline()
    output.append([exclude_sample, float(train_loss), float(train_accuracy), float(val_loss), float(val_accuracy), float(test_loss), float(test_accuracy)])
    
new_df = pd.DataFrame(columns=["exclude_sample", "train_loss", "train_accuracy", "val_loss", "val_accuracy", "test_loss", "test_accuracy"], data=output)

100%|██████████| 24/24 [00:00<00:00, 15185.29it/s]


In [52]:
new_df[["train_accuracy", "val_accuracy", "test_accuracy"]].mean() * 100

train_accuracy    99.993682
val_accuracy      99.766077
test_accuracy     83.034261
dtype: float64

In [53]:
new_df[["train_loss", "val_loss", "test_accuracy"]].sem() * 100

train_loss       0.015288
val_loss         0.089735
test_accuracy    6.756506
dtype: float64

In [83]:
from sklearn.dummy import DummyClassifier

output2 = []

for exclude_sample in tqdm(meta_data.file_name.unique()):
    #
    gc.collect()
    # 
    meta_data_train, meta_data_val = train_test_split(
      meta_data[meta_data.file_name != exclude_sample], test_size=0.2)
    # 
    meta_data_test = meta_data[meta_data.file_name == exclude_sample]
    
    #
    X_train = meta_data_train.drop(["label", "histology", "who_grade"], axis=1)
    Y_train = meta_data_train["label"]
    
    #
    X_test = meta_data_test.drop(["label", "histology", "who_grade"], axis=1)
    Y_test = meta_data_test["label"]
    
    # 
    dummy_clf = DummyClassifier(strategy="most_frequent")
    
    # 
    dummy_clf.fit(X_train, Y_train)
    
    #
    output2.append([dummy_clf.score(X_train, Y_train), dummy_clf.score(X_test, Y_test)])

100%|██████████| 24/24 [00:06<00:00,  3.49it/s]


In [86]:
new_df_2 = pd.DataFrame(columns=["dummy_train_accuracy", "dummy_test_accuracy"], data=output2)

In [88]:
new_df_2[["dummy_train_accuracy", "dummy_test_accuracy"]].mean() * 100

dummy_train_accuracy    63.467914
dummy_test_accuracy     66.666667
dtype: float64

In [89]:
new_df_2[["dummy_train_accuracy", "dummy_test_accuracy"]].sem() * 100

dummy_train_accuracy    0.496154
dummy_test_accuracy     9.829464
dtype: float64